<a href="https://colab.research.google.com/github/mitsosg10/movie_analyzer/blob/main/movie_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start by getting the API token from kaggle

In [2]:
# uploading API token
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mitsosg10","key":"3b34a843fbbc9771edb3fe293259f44e"}'}

In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# Loading csv files
!kaggle datasets download -d rounakbanik/the-movies-dataset
!unzip the-movies-dataset.zip


Dataset URL: https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
License(s): CC0-1.0
 99% 226M/228M [00:02<00:00, 150MB/s]
100% 228M/228M [00:02<00:00, 117MB/s]
Archive:  the-movies-dataset.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


In [6]:
# Cennect to google drive in order to upload the results there, into a json file.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Defining the path
json_file_path='/content/drive/MyDrive/json_files'

# Data overview

In [7]:
# libraries
import pandas as pd
import logging
import json
import ast


In [8]:
# defining csv files
crd=pd.read_csv('credits.csv')
kw=pd.read_csv('keywords.csv')
links=pd.read_csv('links.csv')
sl=pd.read_csv('links_small.csv')
md=pd.read_csv('movies_metadata.csv')
rt=pd.read_csv('ratings.csv')
rts=pd.read_csv('ratings_small.csv')

<ipython-input-8-e7bbb2a5cb74>:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md=pd.read_csv('movies_metadata.csv')


## credits

In [ ]:
# first 5 rows
crd.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [ ]:
# data types
crd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


ID is the only numeric column

In [ ]:
# shape
crd.shape

(45476, 3)

45476 rows and 3 columns

## keywords

In [ ]:
# first 5 rows
kw.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [ ]:
# shape
kw.shape

(46419, 2)

As the shape of csv files is different, we need to create a dataframe containing every csv file.

In [ ]:
kw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


## Links

In [ ]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
len(links.value_counts())

45624

In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  45843 non-null  int64  
 1   imdbId   45843 non-null  int64  
 2   tmdbId   45624 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.0 MB


This file has only numerical columns

In [ ]:
links.shape

(45843, 3)

## Links small

In [ ]:
# first 5 rows
sl.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [ ]:
# data types
sl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [ ]:
sl.shape

(9125, 3)

## Metadata

In [ ]:
# first 5 rows
md.head().T

,0,1,2,3,4
adult,False,False,False,False,False
belongs_to_collection,"{'id': 10194, 'name': 'Toy Story Collection', ...",NaN,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",NaN,"{'id': 96871, 'name': 'Father of the Bride Col..."
budget,30000000,65000000,0,16000000,0
genres,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'id': 35, 'name': 'Comedy'}]"
homepage,http://toystory.disney.com/toy-story,NaN,NaN,NaN,NaN
id,862,8844,15602,31357,11862
imdb_id,tt0114709,tt0113497,tt0113228,tt0114885,tt0113041
original_language,en,en,en,en,en
original_title,Toy Story,Jumanji,Grumpier Old Men,Waiting to Exhale,Father of the Bride Part II
overview,"Led by Woody, Andy's toys live happily in his ...",When siblings Judy and Peter discover an encha...,A family wedding reignites the ancient feud be...,"Cheated on, mistreated and stepped on, the wom...",Just when George Banks has recovered from his ...


In [ ]:
# data types
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

4 numeric(float) variables and 20 categorical.

In [ ]:
md.shape

(45466, 24)

24 columns on this file

## Ratings

In [9]:
# first 5 rows
rt.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [10]:
# data types
rt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 794.2 MB


Only numeric columns

In [ ]:
# shape
rt.shape

(26024289, 4)

As expected on ratings there are many rows, since one user usually rates many movies.

In [ ]:
# # basic statistical measures
rt.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,26024289.0,1.350371e+05,7.817620e+04,1.0,67164.0,1.351630e+05,2.026930e+05,2.708960e+05
movieId,26024289.0,1.584911e+04,3.108526e+04,1.0,1073.0,2.583000e+03,6.503000e+03,1.762750e+05
rating,26024289.0,3.528090e+00,1.065443e+00,0.5,3.0,3.500000e+00,4.000000e+00,5.000000e+00
timestamp,26024289.0,1.171258e+09,2.052889e+08,789652004.0,990754461.0,1.151716e+09,1.357578e+09,1.501830e+09


## Ratings small

In [ ]:
# first 5 rows
rts.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
# data types
rts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [ ]:
# shape
rts.shape

(100004, 4)

In [ ]:
# basic statistical measures
rts.describe().T

,count,mean,std,min,25%,50%,75%,max
userId,100004.0,3.470113e+02,1.951638e+02,1.0,182.0,3.670000e+02,5.200000e+02,6.710000e+02
movieId,100004.0,1.254866e+04,2.636920e+04,1.0,1028.0,2.406500e+03,5.418000e+03,1.639490e+05
rating,100004.0,3.543608e+00,1.058064e+00,0.5,3.0,4.000000e+00,4.000000e+00,5.000000e+00
timestamp,100004.0,1.129639e+09,1.916858e+08,789652009.0,965847824.0,1.110422e+09,1.296192e+09,1.476641e+09


This is not a coplete EDA process, as the task needs only some specific information, links_small and ratings_small is just a subset of links and ratings respectively so they will be ignored.

In [ ]:
# Converting tmdbId into integer
links = pd.read_csv('/content/links.csv', usecols=['tmdbId'],dtype="Int64")

In [ ]:
links.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45843 entries, 0 to 45842
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   tmdbId  45624 non-null  Int64
dtypes: Int64(1)
memory usage: 403.0 KB


# Operations

Top-rated movies might need a different method as a movie with just 10 5/5 ratings could consider the best, but in reality, it is not. So I use the TMDB Ratings to come up with our Top Movies list. I will use IMDB's weighted rating formula to construct my chart. Mathematically, it is represented as follows:

Weighted Rating (WR) =  (vv+m.R)+(mv+m.C)

where,

*   v is the number of votes for the movie
*   m is the minimum number of votes required to be listed in the chart
*   R is the average rating of the movie
*   C is the mean vote across the whole report

The next step is determining an appropriate value for m, the minimum votes required to be listed in the chart. We will use the 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the films in the list. Since many movies have the same title but are released in different year, the list of average ratings will be displayed based on ID.

In [11]:


# Calculate weighted rating
def weighted_rating(x, m, C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

json_file_path = '/content/drive/MyDrive/json_files/results.json'

class MovieAnalyzer:
    """A class to analyze movie data."""

    def __init__(self, csv_files):
        """
        Initialize the MovieAnalyzer with a dictionary of CSV file paths.

        Parameters:
        csv_files (dict): A dictionary containing paths to different CSV files,
                         including 'metadata', 'credits', 'keywords', 'links', and 'ratings'.
                         Each key-value pair represents a DataFrame containing movie data.
        """
        # Initialize custom logger
        self.logger = logging.getLogger('MovieAnalyzerLogger')
        self.logger.setLevel(logging.INFO)
        handler = logging.StreamHandler()
        handler.setLevel(logging.INFO)
        formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)

        self.data = csv_files
        self.md = self.data.get('metadata')
        self.crd = self.data.get('credits')
        self.kw = self.data.get('keywords')
        self.links = self.data.get('links')
        self.rt = self.data.get('ratings')

        self.avgrt = None
        self.gc = None
        self.top5 = None
        self.un = None
        self.mpy = None
        self.df = None

        # Drop the 'movieId' column from links DataFrame if present
        if self.links is not None and 'movieId' in self.links.columns:
            del self.links['movieId']

    def unique_movies_count(self):
        """
        Print the number of unique movies in the dataset.

        If metadata DataFrame is loaded, computes the number of unique movies based on 'id' column
        and stores the result in the 'un' attribute. If metadata is not loaded, logs an error message.
        """
        try:
            if self.md is not None:
                unique_movies = self.md['id'].nunique()
                self.logger.info(f"Number of unique movies: {unique_movies}")
                self.un = unique_movies
            else:
                self.logger.error("Metadata dataset is not loaded.")
        except Exception as e:
            self.logger.error(f"Error calculating unique movies count: {e}")

    def average_rating(self):
        """
        Print the average rating of all the movies.

        Computes the average rating based on 'rating' values in the ratings DataFrame ('self.rt').
        Stores the average ratings in the 'avgrt' attribute as a dictionary mapping movie IDs to their
        corresponding average ratings.
        """
        try:
            if self.rt is not None:
                movie_count = {}
                movie_num = {}
                for movie_id, movie_rt in zip(self.rt['movieId'], self.rt["rating"]):
                    if movie_id:
                        if movie_id not in movie_count:
                            movie_count[movie_id] = movie_rt
                            movie_num[movie_id] = 1
                        else:
                            movie_count[movie_id] += movie_rt
                            movie_num[movie_id] += 1

                avg_rt = {i: movie_count[i] / movie_num[i] for i in movie_num}
                self.avgrt = avg_rt
                self.logger.info("Average ratings computed successfully.")
            else:
                self.logger.error("Ratings dataset is not loaded.")
        except Exception as e:
            self.logger.error(f"Error calculating average ratings: {e}")

    def top_rated_movies(self, n=5):
        """
        Print the top n highest rated movies.

        Calculates a weighted rating ('wr') for each movie based on vote counts and averages.
        Filters and sorts movies based on 'wr', and stores the top 5 movies in the 'top5' attribute.
        """
        try:
            if self.md is not None:
                # Preprocess the genres column
                self.md['genres'] = self.md['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

                # Calculate vote counts and vote averages
                vote_counts = self.md[self.md['vote_count'].notnull()]['vote_count'].astype('int')
                vote_averages = self.md[self.md['vote_average'].notnull()]['vote_average'].astype('int')

                # Calculate C and m
                C = vote_averages.mean()
                m = vote_counts.quantile(0.95)

                # Filter qualified movies
                qualified = self.md[(self.md['vote_count'] >= m) & (self.md['vote_count'].notnull()) & (self.md['vote_average'].notnull())][['title', 'release_date', 'vote_count', 'vote_average', 'popularity', 'genres']]

                qualified['wr'] = qualified.apply(weighted_rating, args=(m, C), axis=1)

                # Sort and find the top 5 movies
                top_5_movies = qualified.sort_values('wr', ascending=False).head(5)
                self.top5 = top_5_movies
                self.logger.info("Top rated movies computed successfully.")
            else:
                self.logger.error("Metadata dataset is not loaded.")
        except Exception as e:
            self.logger.error(f"Error calculating top rated movies: {e}")

    def movies_per_year(self):
        """
        Print the number of movies released each year.

        Converts 'release_date' column in metadata DataFrame ('self.md') to datetime format.
        Computes the count of movies released each year and stores the result in the 'mpy' attribute
        as a DataFrame with columns 'Year' and 'Number of Movies'.
        """
        try:
            if self.md is not None:
                self.md['release_date'] = pd.to_datetime(self.md['release_date'], errors='coerce')
                self.md['release_year'] = self.md['release_date'].dt.year
                movies_per_year = self.md['release_year'].value_counts().sort_index()

                # Convert to DataFrame for better visibility
                movies_per_year_df = movies_per_year.reset_index()
                movies_per_year_df.columns = ['Year', 'Number of Movies']

                self.mpy = movies_per_year_df

                self.logger.info("Number of movies released each year computed successfully.")
            else:
                self.logger.error("Metadata dataset is not loaded.")
        except Exception as e:
            self.logger.error(f"Error calculating movies per year: {e}")

    def movies_per_genre(self):
        """
        Count the number of movies per genre.

        Parses the 'genres' column in the metadata DataFrame ('self.md') to identify different genres
        associated with each movie. Counts the occurrences of each genre and stores the result in the 'gc'
        attribute as a dictionary mapping each genre to its count.
        """
        try:
            genre_count = {}
            if self.md is not None:
                for genres_strings in self.md['genres']:
                    if ast.literal_eval(genres_strings):
                        for genre_dict in ast.literal_eval(genres_strings):
                            genre_name = genre_dict.get('name')
                            if genre_name not in genre_count.keys():
                                genre_count[genre_name] = 1
                            else:
                                genre_count[genre_name] += 1
                self.gc = genre_count
                self.logger.info("Movies per genre computed successfully.")
            else:
                self.logger.error("Metadata dataset is not loaded.")
        except Exception as e:
            self.logger.error(f"Error counting movies per genre: {e}")

    def combine_results(self):
        """
        Combine all analysis results into a single Series.

        Combines the results from various methods (unique_movies_count, average_rating,
        top_rated_movies, movies_per_year, movies_per_genre) into a single pandas Series ('self.df').
        The Series contains the following key-value pairs:
            - 'Unique Movies Count': Number of unique movies in the dataset.
            - 'Average Ratings': Dictionary mapping movie IDs to their average ratings.
            - 'Top Rated Movies': Top 5 movies
            - 'Top Rated Movies': Top 5 movies based on weighted rating.
            - 'Movies Per Year': DataFrame with years and corresponding number of movies released.
            - 'Movies Per Genre': Dictionary mapping each genre to the number of movies in that genre.
        """
        try:
            combined_data = {
                'Unique Movies Count': self.un,
                'Average Ratings': self.avgrt,
                'Top Rated Movies': self.top5.to_dict('records') if self.top5 is not None else None,
                'Movies Per Year': self.mpy.to_dict('records') if self.mpy is not None else None,
                'Movies Per Genre': self.gc
            }
            self.df = pd.Series(combined_data)
            self.logger.info("Results combined into Series successfully.")
        except Exception as e:
            self.logger.error(f"Error combining results into Series: {e}")

    def save_series_to_json(self, json_file_path):
        """
        Save the Series data to a JSON file.

        Converts the Series ('self.df') to a dictionary and saves it as JSON to the specified file path.

        Parameters:
        json_file_path (str): Path to the JSON file where the Series data will be saved.
        """
        try:
            def convert_timestamps(value):
                """
                Convert pandas Timestamp objects to string format for JSON serialization.

                Parameters:
                value: The value to be checked and converted if it's a pandas Timestamp.

                Returns:
                str: The string representation of the Timestamp, or the original value if not a Timestamp.
                """
                if isinstance(value, pd.Timestamp):
                    return value.strftime('%Y-%m-%d %H:%M:%S')
                return value

            if self.df is not None:
                with open(json_file_path, 'w') as f:
                    json.dump(self.df.to_dict(), f, default=convert_timestamps)
                self.logger.info("Results saved to JSON successfully.")
            else:
                self.logger.error("Series data is empty. Nothing to save.")
        except Exception as e:
            self.logger.error(f"Error saving Series to JSON: {e}")

# Define DataFrames
csv_files = {
    'metadata': md,
    'credits': crd,
    'keywords': kw,
    'links': links,
    'ratings': rt
}

# Create an instance of MovieAnalyzer
movie_analyzer = MovieAnalyzer(csv_files)

# Perform operations
movie_analyzer.unique_movies_count()
movie_analyzer.average_rating()
movie_analyzer.movies_per_year()
movie_analyzer.movies_per_genre()
movie_analyzer.top_rated_movies()
movie_analyzer.combine_results()
movie_analyzer.save_series_to_json(json_file_path)


2024-06-15 12:10:33,773 - INFO - Number of unique movies: 45436
INFO:MovieAnalyzerLogger:Number of unique movies: 45436
2024-06-15 12:10:57,059 - INFO - Average ratings computed successfully.
INFO:MovieAnalyzerLogger:Average ratings computed successfully.
2024-06-15 12:10:57,098 - INFO - Number of movies released each year computed successfully.
INFO:MovieAnalyzerLogger:Number of movies released each year computed successfully.
2024-06-15 12:10:59,637 - INFO - Movies per genre computed successfully.
INFO:MovieAnalyzerLogger:Movies per genre computed successfully.
2024-06-15 12:11:02,253 - INFO - Top rated movies computed successfully.
INFO:MovieAnalyzerLogger:Top rated movies computed successfully.
2024-06-15 12:11:02,265 - INFO - Results combined into Series successfully.
INFO:MovieAnalyzerLogger:Results combined into Series successfully.
2024-06-15 12:11:02,943 - INFO - Results saved to JSON successfully.
INFO:MovieAnalyzerLogger:Results saved to JSON successfully.


In [12]:
movie_analyzer.gc

{'Animation': 1935,
 'Comedy': 13182,
 'Family': 2770,
 'Adventure': 3496,
 'Fantasy': 2313,
 'Romance': 6735,
 'Drama': 20265,
 'Action': 6596,
 'Crime': 4307,
 'Thriller': 7624,
 'Horror': 4673,
 'History': 1398,
 'Science Fiction': 3049,
 'Mystery': 2467,
 'War': 1323,
 'Foreign': 1622,
 'Music': 1598,
 'Documentary': 3932,
 'Western': 1042,
 'TV Movie': 767,
 'Carousel Productions': 1,
 'Vision View Entertainment': 1,
 'Telescene Film Group Productions': 1,
 'Aniplex': 1,
 'GoHands': 1,
 'BROSTA TV': 1,
 'Mardock Scramble Production Committee': 1,
 'Sentai Filmworks': 1,
 'Odyssey Media': 1,
 'Pulser Productions': 1,
 'Rogue State': 1,
 'The Cartel': 1}

In [13]:
movie_analyzer.avgrt

{110: 4.016057252826558,
 147: 3.5959331588483994,
 858: 4.339810758717364,
 1221: 4.263475012950189,
 1246: 3.9128028890959925,
 1968: 3.827552515876893,
 2762: 4.021281953145459,
 2918: 3.9269283300984834,
 2959: 4.2307160469145675,
 4226: 4.1570775807006335,
 4878: 3.9725600543888806,
 5577: 3.5524292205522543,
 33794: 3.944855598774569,
 54503: 3.73027474462839,
 58559: 4.182070707070707,
 59315: 3.855209887065843,
 68358: 3.8723525852215426,
 69844: 3.8431253785584496,
 73017: 3.7661114570361147,
 81834: 3.8732918180245632,
 91500: 3.5621421650665024,
 91542: 3.738952195664258,
 92439: 3.3820754716981134,
 96821: 3.8701923076923075,
 98809: 3.7040406295345463,
 99114: 4.00847806888431,
 112552: 4.120283855706682,
 5: 3.079564818455892,
 25: 3.6675128965433204,
 32: 3.888769242293719,
 58: 3.965010863864539,
 64: 2.8375228519195614,
 79: 3.1614385297845375,
 141: 3.5228261714196907,
 260: 4.132299305600623,
 339: 3.5089935090326114,
 377: 3.4874077814860356,
 605: 3.308572179858007

In [14]:
movie_analyzer.top5

,title,release_date,vote_count,vote_average,popularity,genres,wr
314,The Shawshank Redemption,1994-09-23,8358.0,8.5,51.645403,"[Drama, Crime]",8.339318
834,The Godfather,1972-03-14,6024.0,8.5,41.109264,"[Drama, Crime]",8.281246
12481,The Dark Knight,2008-07-16,12269.0,8.3,123.167259,"[Drama, Action, Crime, Thriller]",8.195622
2843,Fight Club,1999-10-15,9678.0,8.3,63.869599,[Drama],8.168877
292,Pulp Fiction,1994-09-10,8670.0,8.3,140.950236,"[Thriller, Crime]",8.154359


Based on the weighted rating, The Shawshank Redemption is the top-rated movie on the dataset.

In [15]:
movie_analyzer.un

45436

In [16]:
movie_analyzer.mpy

,Year,Number of Movies
0,1874.0,1
1,1878.0,1
2,1883.0,1
3,1887.0,1
4,1888.0,2
...,...,...
130,2015.0,1905
131,2016.0,1604
132,2017.0,532
133,2018.0,5


From 2005 to 2016 more than 1000 movies released each year.

In [17]:
print(movie_analyzer.df)

Unique Movies Count                                                45436
Average Ratings        {110: 4.016057252826558, 147: 3.59593315884839...
Top Rated Movies       [{'title': 'The Shawshank Redemption', 'releas...
Movies Per Year        [{'Year': 1874.0, 'Number of Movies': 1}, {'Ye...
Movies Per Genre       {'Animation': 1935, 'Comedy': 13182, 'Family':...
dtype: object
